In [1]:
import numpy as np
import pandas as pd

TEST_SIZE = 1000
MAX_VALUE = 255 # should be for pixels (grayscale values range from 0-255 and 87 for musical notes (notes range from 0-87))
COL = 784
RESULT_SIZE = 10

df = pd.read_csv('number_data.csv') # can be any of the 4 datasets

In [2]:
data = np.array(df)
np.random.shuffle(data) # randomize order of data
r, c = data.shape

data_dev = data[:TEST_SIZE].T
Y_dev = data_dev[0]
X_dev = data_dev[1:]
X_dev = X_dev / MAX_VALUE

data_train = data[TEST_SIZE:].T
Y_train = data_train[0]
X_train = data_train[1:]
X_train = X_train / MAX_VALUE
_,m_train = X_train.shape

In [3]:
def init_params():
    w1 = np.random.rand(RESULT_SIZE, COL) - 0.5
    c1 = np.random.rand(RESULT_SIZE, 1) - 0.5
    w2 = np.random.rand(RESULT_SIZE, RESULT_SIZE) - 0.5
    c2 = np.random.rand(RESULT_SIZE, 1) - 0.5
    return w1, c1, w2, c2

def ReLU(n): # 1st activation function (x if x>0, 0 else)
    return np.maximum(n, 0)

def softmax(n): # 2nd activation function
    return np.exp(n) / sum(np.exp(n))
    
def for_prop(w1, c1, w2, c2, X): #forward propagation
    L1 = w1.dot(X) + c1
    R1 = ReLU(L1)
    L2 = w2.dot(R1) + c2
    R2 = softmax(L2)
    return L1, R1, L2, R2

def ReLU_deriv(n):
    return 1 if n > 0 else 0

def res(Y):
    res_Y = np.zeros((Y.size, Y.max() + 1))
    res_Y[np.arange(Y.size), Y] = 1
    res_Y = res_Y.T
    return res_Y

def back_prop(L1, R1, R2, w2, X, Y):
    res_Y = res(Y)
    dL2 = R2 - res_Y
    dw2 = 1 / r * dL2.dot(R1.T)
    dc2 = 1 / r * np.sum(dL2)
    dL1 = w2.T.dot(dL2) * ReLU_deriv(L1)
    dw1 = 1 / r * dL1.dot(X.T)
    dc1 = 1 / r * np.sum(dL1)
    return dw1, dc1, dw2, dc2

def update(w1, c1, w2, c2, dw1, dc1, dw2, dc2, speed):
    w1 = w1 - speed * dw1
    c1 = c1 - speed * dc1    
    w2 = w2 - speed * dw2  
    c2 = c2 - speed * dc2    
    return w1, c1, w2, c2

In [4]:
def predict(R2):
    return np.argmax(R2, 0)

def check(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, speed, iterations):
    w1, c1, w2, c2 = init_params()
    for i in range(iterations):
        L1, R1, L2, R2 = for_prop(w1, c1, w2, c2, X)
        dw1, dc1, dw2, dc2 = back_prop(L1, R1, R2, w2, X, Y)
        w1, c1, w2, c2 = update(w1, c1, w2, c2, dw1, dc1, dw2, dc2, speed)
        if i % 500 == 0:
            print("Iteration: ", i)
            predictions = predict(A2)
            print(check(predictions, Y))
    return w1, c1, w2, c2

def make_predictions(X, w1, c1, w2, c2):
    _, _, _, A2 = for_prop(w1, c1, w2, c2, X)
    predictions = predict(A2)
    return predictions

def test_prediction(index, w1, c1, w2, c2):
    prediction = make_predictions(X_train[:, index, None], w1, c1, w2, c2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)

In [5]:
w1, c1, w2, c2 = gradient_descent(X_train, Y_train, 0.2, 2000)

Iteration:  0
[1 2 2 ... 0 2 2] [1 0 5 ... 3 6 8]
0.20014634146341465
Iteration:  100
[1 0 0 ... 3 6 5] [1 0 5 ... 3 6 8]
0.7471463414634146
Iteration:  200
[1 0 0 ... 3 6 5] [1 0 5 ... 3 6 8]
0.8221463414634146
Iteration:  300
[1 0 0 ... 3 6 5] [1 0 5 ... 3 6 8]
0.8539512195121951
Iteration:  400
[1 0 0 ... 3 6 7] [1 0 5 ... 3 6 8]
0.8691219512195122
Iteration:  500
[1 0 2 ... 3 6 7] [1 0 5 ... 3 6 8]
0.8758292682926829
Iteration:  600
[1 0 3 ... 3 6 7] [1 0 5 ... 3 6 8]
0.8884390243902439
Iteration:  700
[1 0 3 ... 3 6 7] [1 0 5 ... 3 6 8]
0.8987317073170732
Iteration:  800
[1 0 3 ... 3 6 7] [1 0 5 ... 3 6 8]
0.8948780487804878
Iteration:  900
[1 0 2 ... 3 6 7] [1 0 5 ... 3 6 8]
0.8897073170731707
Iteration:  1000
[1 0 7 ... 3 6 7] [1 0 5 ... 3 6 8]
0.8923658536585366
Iteration:  1100
[1 0 7 ... 3 6 7] [1 0 5 ... 3 6 8]
0.9065853658536586
Iteration:  1200
[1 0 7 ... 3 6 7] [1 0 5 ... 3 6 8]
0.9063170731707317
Iteration:  1300
[1 0 7 ... 3 6 7] [1 0 5 ... 3 6 8]
0.9118048780487805
Ite

In [9]:
dev_predictions = make_predictions(X_dev, w1, c1, w2, c2)
check(dev_predictions, Y_dev)

[5 9 3 1 2 8 2 2 3 4 8 2 1 9 3 3 0 0 6 2 7 3 6 4 0 8 4 0 2 4 2 7 6 1 8 2 7
 7 1 1 6 2 4 5 6 4 3 8 0 1 6 7 0 9 8 6 6 3 0 1 0 5 2 8 4 0 7 2 2 3 1 0 1 1
 1 5 8 2 7 4 8 7 6 0 0 6 7 4 9 3 5 2 3 4 4 3 4 9 6 5 5 4 4 3 3 2 3 9 0 6 9
 4 7 4 5 2 7 7 1 8 6 6 0 1 8 6 9 3 8 0 6 3 3 1 1 5 5 2 0 9 0 9 7 2 4 9 7 2
 6 5 2 8 9 5 2 2 3 0 9 5 3 6 3 0 3 4 7 3 1 5 3 9 6 2 5 1 7 0 8 5 5 2 0 9 4
 9 0 9 1 9 8 3 6 9 0 5 6 3 2 4 6 5 9 2 1 2 8 7 2 7 5 7 8 7 4 8 5 6 0 2 5 4
 5 1 9 1 9 3 0 4 8 9 4 2 5 2 2 2 5 4 6 6 0 1 7 5 2 9 5 1 1 2 6 4 1 3 0 1 2
 3 9 3 4 2 5 6 9 7 6 3 3 0 0 1 0 3 3 2 7 2 1 3 9 5 0 4 0 9 6 0 9 7 7 7 5 5
 3 2 1 6 1 2 5 5 8 8 5 0 7 2 9 7 9 9 6 1 7 3 4 4 5 1 7 9 5 0 6 0 6 7 9 0 5
 1 5 4 2 3 6 2 3 7 3 8 6 8 4 2 1 5 0 0 3 5 8 3 9 9 3 9 2 6 9 2 5 6 7 8 9 9
 4 7 2 0 3 5 8 5 4 9 3 6 5 2 7 3 3 5 3 9 7 2 9 2 9 1 1 8 0 2 9 7 6 8 5 2 4
 9 4 0 5 3 4 6 1 9 0 1 9 3 8 4 8 7 9 2 4 6 4 9 8 2 9 5 9 6 3 2 2 7 1 6 4 4
 8 5 9 2 5 9 9 3 6 5 3 6 1 6 9 1 6 3 7 3 9 6 3 4 9 2 6 6 3 9 1 8 1 4 0 1 1
 8 6 5 0 4 8 9 1 0 0 5 9 

0.937